<a href="https://colab.research.google.com/github/Ni-cole17/Diagnostic-ECG-Database/blob/main/Prepare_and_Visualize_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wfdb
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 161 kB 5.2 MB/s 


In [3]:
import wfdb
import pandas as pd
import numpy as np
import re
import csv
import os
import json 
from itertools import islice
import ast
import sys
import matplotlib.pyplot as plt
import plotly.express as px
import random
import tqdm

## Extract and Organize Data

In [4]:
# Go to your directory path
# If you don't have a directory run the git clone cell
%cd '/content/drive/MyDrive/Projeto-Tecnicas'

/content/drive/MyDrive/Projeto-Tecnicas


In [ ]:
#!git clone https://github.com/Ni-cole17/Diagnostic-ECG-Database.git

### Generate Json of heas

In [ ]:
# Just run this if you want to generate another heas.json
# Copying zip database to colab content

#!cp '/content/drive/MyDrive/Projeto-Tecnicas/ptb-diagnostic-ecg-database-1.0.0.zip' /content

# Unzip Database
#!7z x '/content/ptb-diagnostic-ecg-database-1.0.0.zip'

In [ ]:
path_orig = '/content/ptb-diagnostic-ecg-database-1.0.0'
paths = [paths for paths in os.listdir(path_orig) if os.path.isdir(os.path.join(path_orig,paths)) == True]
patient_heas = {}
for i in paths:
  list_hea = [hea for hea in os.listdir(os.path.join(path_orig,i)) if hea.endswith('.hea')]
  patient_heas[i] = {'heas': list_hea}

print(patient_heas)

In [ ]:
with open("patient_heas.json", "w") as outfile:
    json.dump(patient_heas, outfile,indent = 4)

### Openning json and import classes

In [5]:
# Openning heas json
heas_json = dict(pd.read_json('patient_heas.json',typ='series'))

In [8]:
# Importing Classes from src folder
from src.ReadFile import ReadFile
from src.OrganizeFields import OrganizeFields
from src.Registro import Registro

### Generate Data

In [9]:
# Generating cvs with record and signal information and Generating .txt files
# You have to have a folder on your directory called Signals (this is where will save the .txt files)

# Walking on json items
for k,v in tqdm.tqdm(heas_json.items()):
  new_patient = True 

  # Id Patient
  idP = k
  for value in v['heas']:
    # Id Signal
    idS = value.replace('.hea','')

    # Reading signal and fields from wfdb
    signal, fields = ReadFile(idS,k).read()

    # Getting organized dicts and saving the data
    fields,signal_fields = OrganizeFields(fields).get_dicts()
    # Descomment the code below with you want to save data
    ##register = Registro(fields,signal_fields,signal,new_patient,idP,idS).save_data()

    # Setting new patient False, because we don't want save the same patient twice on Record_info.csv
    new_patient = False

100%|██████████| 290/290 [04:51<00:00,  1.01s/it]


## Visualization


In [116]:
class ItemsSignal:
  def __init__(self,base,idP,idR,signal):
    self.base = base
    self.idP = idP
    self.idR = idR
    self.signal = signal

  def Query(self,base):

    return base[(base.idP == self.idP) & (base.idS == self.idS)]

  def take_fs(self):
    select = self.Query()
    fs = int(pd.Series(select.fs).values)
    return fs

  def take_time(self):
    fs = self.take_fs()
    sizesig = len(self.signal)
    ts = 1 / fs
    t = np.arange(0, sizesig) * ts

    return t

  def take_signal(self):
    signal = self.Query(self.signal)
    signal = str(pd.Series(signal.Signal).values)
    record = record = wfdb.rdrecord(signal, channels=[*range(15)], sampfrom=100, sampto=15000) 
    record =record.__dict__
    array_sig = record['p_signal']
    return array_sig

In [117]:
class Visualization:
  def __init__(self,csv,idP,idR,signal):
   self.base = pd.read_csv(csv)
   self.idP = idP
   self.idR = idR
   self.signal = signal  # serÃƒÂ¡ um dataframe igual o antigo mas em vez de ser o array serÃƒÂ¡ o caminho do arquivo dat
   self.Items = ItemsSignal(self.base,self.idP,self.idR,self.signal)
    
  def Hist(self,column):
    plt.hist(self.base.column)

  def plot_signal(self,Type):
    t = self.Items.take_time()
    sig = self.Items.take_signal()

    if Type ==  'all':
      figure, ax1 = plt.subplots(1, 1,figsize=(15,7))
      ax1.plot(t, sig, linewidth=2)
      ax1.set_title("Record " + str(self.idR) + ' Pacient ' + str(self.idP))
      ax1.set_xlabel("Time(s)")
      ax1.set_ylabel("Amplitude")
      ax1.grid()
      plt.figure (figsize=(18,9))

    elif Type == 'channels':
      num_of_channels = self.signal.shape[1]
      cs = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(num_of_channels)]
      plt.figure (figsize=(30,15))
      count = 0
      for i in range(num_of_channels):
        current_color=cs[count]
        plt.plot(t,self.signal[::,i]+(num_of_channels-count)*1,color=current_color,linewidth=1,label = 'channel'+ str(i))
        plt.legend(labelspacing = 3)
        count = count + 1
      plt.tick_params(axis='y', which='minor', labelsize=10,pad = 10)
      plt.title("Record " + str(self.idR) + 'Pacient ' + str(self.idP))
      plt.xlabel("Time(s)")
      plt.ylabel("Amplitude")
      plt.tight_layout()

  def up_plot(self):
    signal = self.Items.take_signal()
    plt.figure(figsize=(18,9))
    plt.imshow(signal.transpose(),aspect = 'auto',cmap= 'jet_r',origin = 'upper',resample= True)
    plt.title("Record " + str(self.idR) + 'Pacient ' + str(self.idP))
    plt.colorbar()

  def Vetocardiogram(self):
    signal = self.Items.take_signal()
    vx = signal[:,12]
    vy = signal[:,13]
    vz = signal[:,14]
    df = pd.DataFrame({"x": vx, "y":vy, "z":vz})
    fig = px.line_3d(df,x ='x',y = 'y',z = 'z',title = "Record vectorcardiogram " + str(self.idR) + 'Pacient ' + str(self.idP))
    fig.show()